In [62]:
import numpy as np
import pandas as pd
import glob

import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

In [2]:
def zs_data(start: str, end: str, src: str) -> pd.DataFrame:
    """Get data from """
    data_df = pd.read_csv(src, index_col=0)#, parse_dates=["Time"])

    freq='15min'
    intervals_per_hour = 4

    # temp bandaid to deal with substation data
    # data_df.index = pd.to_datetime(data_df.index, utc=True)#, format='mixed')

    # NOTE timestamps indicate start of an interval
    # Starting with 15 min data we'll set the time index to start at the beninning of this 
    # period and then trim off the first 5 min after ffill()
    tmp = pd.date_range(start=pd.to_datetime("2021-7-1 00:00").tz_localize("UTC"), 
                        end=pd.to_datetime("2023-06-30 23:45").tz_localize("UTC"), 
                        freq=freq)
    data_df.index = tmp

    # data_df = data_df.resample("5min").ffill()#.reset_index()

    # data_df.rename(columns={data_df.columns[0]: "network_load"}, inplace=True)

    new_df = data_df.loc[start:end]#[1:]

    # new_df.reset_index(inplace=True)

    # temp bandaid to deal with substation data
    # new_df = new_df.rename(columns={data_df.index.name:'settlementdate'})
    # new_df = new_df.rename(columns={'index':'settlementdate'})
    return new_df, intervals_per_hour

In [96]:
zs_df, intervals_per_hour = zs_data("2022-1-1 00:00", "2022-12-31 23:45",
    src = "data/evoenergy-zone-substation-report-2021-23.csv")

pop_SA3_df = pd.read_csv('data/SA3_population.csv', index_col=0)
pop_SA2_df = pd.read_csv('data/SA2_population.csv', index_col=0)
evo_cust_df = pd.read_csv('data/evo_customers.csv', index_col=0)

# http://www.bom.gov.au/climate/averages/tables/cw_070282.shtml
mean_3pm_temps = [26.9,26.4,23.5,19.1,14.9,11.4,10.6,12.6,15.1,18.3,22.0,24.8,18.8]
mean_temps_Bin = [20.495001,20.325001,17.85,13.48,9.469999,6.67,5.585,7.105,9.710001,12.985,15.895,18.775]

NextGen_solar = pd.read_csv('data/NextGen_solar.csv', index_col=0) 
NextGen_solar_capacity = NextGen_solar/NextGen_solar.min()
NextGen_solar_capacity.index = zs_df.index

SA3_dict = {'Gungahlin':['Gold Creek'],
            'Tuggeranong': ['Wanniassa', 'Gilmore', 'Theodore'],
            'Woden Valley':['Woden', 'Wanniassa'],
            'Canberra East':['East Lake'],#, 'Fyshwick'],
            'South Canberra':['Telopea Park'],
            'Belconnen':['Belconnen', 'Latham'],
            'North Canberra':['Civic', 'City East'],
            'Weston Creek':['Woden'],
            'Molongolo':['Woden'],
            'Urriarra - Namadgi':['Tennent/Angle Crossing'],}

SA3_division = {'Woden Valley':[39238/(39238+24315+12102), 39238/(39238+88965)],
                'Tuggeranong':[88965/(39238+88965), 1, 1],
            'Weston Creek':[24315/(39238+24315+12102), ],
            'Molongolo':[12102/(39238+24315+12102)],
            'Gungahlin':[1],
            'Canberra East':[1,1],
            'South Canberra':[1],
            'Belconnen':[1,1],
            'North Canberra':[1,1]}

zs_ratings = {'Gold Creek':[57,74], 
             'Gilmore':[45,62],  
             'Theodore':[45,62], 
            'Woden':[95,95], 
            'East Lake':[50,60], 
            'Fyshwick':[28,28], 
            'Telopea Park':[100,114], 
            'Belconnen':[55,74], 
            'Latham':[95,95], 
            'Civic':[110,114],
            'City East':[95,95],
            'Wanniassa':[95,95]} # Table 54  [Continuous,2hr emergency] ratings

ACT_vehicles = 318000 # 2021 data https://www.abs.gov.au/statistics/industry/tourism-and-transport/motor-vehicle-census-australia/latest-release

In [92]:
# ng_demand = pd.DataFrame()
# ng_solar = pd.DataFrame()

# nextgen_files = glob.glob('/home/bjorn/Data/NextGen_csv/*.csv')
# for f in nextgen_files:
#     ng = pd.read_csv(f)
#     ng = ng.set_index('original index')
#     load = ng['load power (kW)'].to_frame(name=f)
#     solar_tmp = ng['solar power (kW)'].to_frame(name=f)
#     ng_demand = pd.concat([ng_demand,load], axis=1)
#     ng_solar = pd.concat([ng_solar,solar_tmp], axis=1)


# # ng_demand_av = ng_demand.mean(axis=1)
# ng_solar_av = ng_solar.mean(axis=1)

# # rearrange to match WW day of year
# # start_time = 1539435600 # 14/10/2018
# # end_time = 1531663200 #16/7/2018
# start_time = 1540990800 # 1/11/2018
# end_time = 1533045600 #1/8/2018
# end_2018_year = 1546261200
# start_2018_year = 1514725200
# # rearrange_year_demand = pd.concat([ng_demand_av.loc[start_time:end_2018_year],
# #     ng_demand_av.loc[start_2018_year:end_time]], axis=0)
# # rearrange_year_solar = pd.concat([ng_solar_av.loc[start_time:end_2018_year],
# #     ng_solar_av.loc[start_2018_year:end_time]], axis=0)

# # rearrange_year_demand = ng_demand_av.loc[start_2018_year:end_2018_year]
# rearrange_year_solar = ng_solar_av.loc[start_2018_year:end_2018_year]

# ng_solar_15min = rearrange_year_solar.groupby(np.arange(len(rearrange_year_solar.index)) // 3).mean()
# ng_solar_15min.index = zs_df.index
# ng_solar_15min.to_csv('NextGen_solar.csv', index=True) 

In [169]:
def forecast(evo_regions, evo_fractions, nu_days, start_day, zs_df, timed_hw_start, timed_hw_end,
             current_households, future_households, mean_temps, #resist_kWh_per_household,
             current_frac_gas, future_frac_gas, current_frac_heatpumps, future_frac_heatpumps,
             future_solar_MW, intervals_per_hour):
    current_frac_elec = 1 - current_frac_gas
    future_frac_elec = 1 - future_frac_gas
    current_frac_resist = 1 - current_frac_heatpumps
    future_frac_resist = 1 - future_frac_heatpumps

    date_dict = {}
    for i_d, day in enumerate(range(start_day,start_day+nu_days)):
        for e, zs in enumerate(evo_regions):
            if e == 0:
                current_day_zs_df = zs_df[zs][zs_df.index.dayofyear == day+1]*evo_fractions[e]
                current_day_zs_df.rename('Current', inplace=True)
            else:
                tmp_df = zs_df[zs][zs_df.index.dayofyear == day+1]*evo_fractions[e]
                tmp_df.rename('Current', inplace=True)
                # print(current_day_zs_df)
                # print(tmp_df)
                current_day_zs_df = current_day_zs_df.add(tmp_df)
                # print(current_day_zs_df)
        # current_day_zs_df.index = current_day_zs_df.index.strftime('%H:%M')
        # current_day_zs_df.index = current_day_zs_df.index.time
        # time_interval = current_day_zs_df.index[1] - current_day_zs_df.index[0]
        # print(current_day_zs_df)
        # print(repr(current_day_zs_df.index))

        date_dict[day] = current_day_zs_df.index[0].date()
        month = current_day_zs_df.index[0].month
        temp = mean_temps[month-1]

        current_gas_users = current_frac_gas*current_households
        future_elec_users = future_households*future_frac_elec
        
        pop_growth_factor = (future_households)/(current_households)
        
        # heating energy demand as function of temperature per household
        CBR_factor = 1.266
        hw_kWh_household_per_day = (10.295 - 0.2816*temp)*CBR_factor
        # Heat pump coefficient of performance as function of temperature
        heatpump_CoP = 2.9664 + 0.0703*temp

        current_hw_MWh_per_day_elec = (current_frac_resist*hw_kWh_household_per_day + \
                                       current_frac_heatpumps*hw_kWh_household_per_day/heatpump_CoP)/1000
        current_hw_MWh_per_day = current_frac_elec*current_households*current_hw_MWh_per_day_elec
        current_hw_MW = current_hw_MWh_per_day/24
        # print('current_hw_kWh', current_hw_MWh_per_day_elec*1000)
            
        curent_day_zs_sans_hw = current_day_zs_df - current_hw_MW
        curent_day_zs_sans_hw.rename('Current w/o hot water', inplace=True)


        ### Future 
        future_solar_day_MW = future_solar_MW[future_solar_MW.index.dayofyear == day+1]

        # future_zs = current_day_zs_df*pop_growth_factor
        future_zs_sans_hw = curent_day_zs_sans_hw*pop_growth_factor - future_solar_day_MW['0']
        
        future_hw_resist_kWh = future_frac_resist*hw_kWh_household_per_day
        future_hw_heatpump_kWh = future_frac_heatpumps*hw_kWh_household_per_day/heatpump_CoP
        future_hw_MWh_total = future_elec_users*(future_hw_heatpump_kWh + future_hw_resist_kWh)/1000
        # print('future_hw_kWh', future_hw_MWh_total/future_elec_users*1000)

        if timed_hw_start < timed_hw_end:
            time_mask = (current_day_zs_df.index.hour >= timed_hw_start) & \
                    (current_day_zs_df.index.hour < timed_hw_end)
        else:
            time_mask = (current_day_zs_df.index.hour >= 0) & \
                    (current_day_zs_df.index.hour < timed_hw_end) + \
                    (current_day_zs_df.index.hour >= timed_hw_start) & \
                    (current_day_zs_df.index.hour <= 24)

        timed_hw_df = pd.Series(index=current_day_zs_df.index, data = 0)
        # timed_hw_df[timed_hw_df.index.hour > timed_hw_start & timed_hw_df.index.hour < timed_hw_end] == 1
        # print(timed_hw_df.between_time(timed_hw_start,timed_hw_end))
        timed_hw_df[time_mask] = 1
        future_hw_MW = future_hw_MWh_total / (np.sum(timed_hw_df)/intervals_per_hour)
        timed_hw_df = timed_hw_df*future_hw_MW
        # print(timed_hw_df)

        future_zs_with_hw = future_zs_sans_hw + timed_hw_df
        future_zs_with_hw.rename('Future', inplace=True)

        # print(future_zs_with_hw)

        plotting_df = pd.concat([current_day_zs_df, curent_day_zs_sans_hw, future_zs_with_hw],axis=1)

        current_day_zs_df.index = current_day_zs_df.index.time
        curent_day_zs_sans_hw.index = current_day_zs_df.index
        future_zs_sans_hw.index = current_day_zs_df.index
        future_zs_with_hw.index = current_day_zs_df.index

        if i_d == 0:
            many_day_df_current = current_day_zs_df.rename(date_dict[day], inplace=True).to_frame()
            many_day_df_current_sans = curent_day_zs_sans_hw.rename(date_dict[day], inplace=True).to_frame()
            many_day_df_future = future_zs_with_hw.rename(date_dict[day], inplace=True).to_frame()
            many_day_df_future_sans = future_zs_sans_hw.rename(date_dict[day], inplace=True).to_frame()
        else:
            many_day_df_current = many_day_df_current.join(current_day_zs_df.rename(date_dict[day]))
            many_day_df_current_sans = many_day_df_current_sans.join(curent_day_zs_sans_hw.rename(date_dict[day]))
            many_day_df_future = many_day_df_future.join(future_zs_with_hw.rename(date_dict[day]))
            many_day_df_future_sans = many_day_df_future_sans.join(future_zs_sans_hw.rename(date_dict[day]))

    plotting_df_many = pd.concat([many_day_df_current.mean(axis=1), many_day_df_current_sans.mean(axis=1), 
                                  many_day_df_future_sans.mean(axis=1), many_day_df_future.mean(axis=1)],axis=1)
    plotting_df_many.columns = ['Current','Current w/o hot water','Future w/o hot water','Future w hot water']

    return many_day_df_current, many_day_df_current_sans, many_day_df_future, \
            many_day_df_future_sans, plotting_df_many, plotting_df, future_hw_MWh_total

In [182]:
future_year = 2045
POE = 0.9
nu_days = 30
start_day = 5*30

timed_hw_start = 9
timed_hw_end = 17
# timed_hw_start = 21
# timed_hw_end = 17
# resist_kWh_per_household = 8.1
current_frac_gas = 0.45
future_frac_gas = 0.0
current_frac_heatpumps = 0.1
future_frac_heatpumps = 0.75

future_solar_capacity_MW = 50
future_solar_MW = future_solar_capacity_MW*NextGen_solar_capacity

SA_level = 3
SA_region = 'Belconnen'
# SA_region = 'Tuggeranong'
# SA_region = 'Gungahlin'
# SA_region = 'Woden Valley'
# SA_region = 'Canberra East'
# SA_region = 'South Canberra'
# SA_region = 'North Canberra'

current_households = 0
zs_combined_rating = 0
for e, r in enumerate(SA3_dict[SA_region]):
    zs_frac = SA3_division[SA_region][e]
    current_households += evo_cust_df.loc[r,"Evo resi customers"]*zs_frac
    zs_combined_rating += zs_ratings[r][0]*zs_frac # Continuous rating

if SA_level == 2:
    current_pop = pop_SA2_df.loc[SA_region,'2022']
    pop_per_household = current_pop/current_households
    # print(pop_per_household)
    future_households = pop_SA2_df.loc[SA_region,str(future_year)]/pop_per_household
elif SA_level == 3:
    current_pop = pop_SA3_df.loc[SA_region,'2022']
    pop_per_household = current_pop/current_households
    future_households = pop_SA3_df.loc[SA_region,str(future_year)]/pop_per_household

# mean_temps = mean_3pm_temps
mean_temps = mean_temps_Bin


many_day_df_current, many_day_df_current_sans, many_day_df_future, \
    many_day_df_future_sans, plotting_df_many, plotting_df, future_hw_MWh_total = \
        forecast(SA3_dict[SA_region], SA3_division[SA_region], nu_days, start_day, zs_df, timed_hw_start, timed_hw_end,
             current_households, future_households, mean_temps, #resist_kWh_per_household,
             current_frac_gas, future_frac_gas, current_frac_heatpumps, future_frac_heatpumps, future_solar_MW,
             intervals_per_hour)


In [183]:

# future_zs_with_hw.plot()
# many_day_df_current.plot()
# many_day_df_current.mean(axis=1).plot()
# plotting_df_many.plot()
fig = go.Figure(plotting_df_many.plot())
fig.update_yaxes(range = [0,zs_combined_rating])
fig.add_hline(y=zs_combined_rating, line_width=3, line_dash="dash", line_color="black")


In [110]:
# Utilization
total_day_profile = many_day_df_current.mean(axis='columns')
total_day_profile_peak = total_day_profile.max()
current_tot_energy = total_day_profile.sum()/intervals_per_hour
total_day_profile = many_day_df_future.mean(axis='columns')
total_day_profile_peak = total_day_profile.max()
future_tot_energy = total_day_profile.sum()/intervals_per_hour

POE_index = int(POE*nu_days)
current_peak_load_day = many_day_df_current.max().idxmax()
current_peak_load_MW = many_day_df_current[current_peak_load_day].max()
# peak_load_MW = many_day_df_future.max(axis='columns')
future_peak_load_day = many_day_df_future.max().idxmax()
future_peak_load_MW = many_day_df_future[future_peak_load_day].max()
sorted_load_days = many_day_df_future.max().sort_values(ascending=True)

high_load_days = sorted_load_days.iloc[POE_index:].index.to_numpy()
many_day_df_future[high_load_days].plot()

min_threshold_POE = many_day_df_future[high_load_days].max().min()


print('Current total energy delivered = ', np.round(current_tot_energy,0), " MWh")
print('Current substation utilisation = ', np.round(100*current_tot_energy/(zs_combined_rating*24),0), " %")
print('Future total energy delivered = ', np.round(future_tot_energy,0), " MWh")
print('Current substation utilisation = ', np.round(100*future_tot_energy/(future_peak_load_MW*24),0), " %")

print(f"Future threshold of {POE} POE = {np.round(min_threshold_POE,1)} MW")
print(f"Future max peak = {np.round(future_peak_load_MW,1)} MW")
print(f"Future HW demand = {np.round(future_hw_MWh_total,1)} MWh/day")
print(f"Future HW demand = {np.round((future_hw_MWh_total/future_households)*1000,1)} kWh/household/day")
print(f"Future HW demand = {np.round((future_hw_MWh_total/future_households)*250000*1000,1)} MWh for 250,000 households/day")
print(f"Current households = {np.round((current_households)/1000,1)} k")
print(f"Future households = {np.round((future_households)/1000,1)} k")

Current total energy delivered =  1018.0  MWh
Current substation utilisation =  54.0  %
Future total energy delivered =  1067.0  MWh
Current substation utilisation =  63.0  %
Future threshold of 0.9 POE = 68.1 MW
Future max peak = 70.0 MW
Future HW demand = 107.0 MWh/day
Future HW demand = 5.0 kWh/household/day
Future HW demand = 1247559.0 MWh for 250,000 households/day
Current households = 20.3 k
Future households = 21.4 k


In [59]:
future_tot_energy

np.float64(128.03377929073554)

In [ ]:
future_households

np.float64(21437.57213664058)

In [ ]:

# fig = make_subplots(rows=1, cols=1)#,specs=[[{"secondary_y": True}]])
fig = go.Figure(total_day_profile.plot())
fig.update_yaxes(range = [0,total_day_profile_peak])